## <center> Packages & Connection </center>

In [15]:
import h3 as h3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy import stats
from pyhive import presto
from keplergl import KeplerGl
from datetime import datetime, timedelta

import warnings
warnings.filterwarnings('ignore')

In [16]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

## <center> Local extracted files </center>

In [17]:
refined_hex_8_all_data = pd.read_csv('hex_8_all_data_2023_07_10_to_16.csv')
refined_hex_8_hsr_yesh = pd.read_csv('hex_8_data_2023_07_10_to_16.csv')

In [18]:
print(refined_hex_8_all_data.shape)
print(refined_hex_8_hsr_yesh.shape)

(4032, 29)
(30, 29)


In [19]:
hex_8_all_data = refined_hex_8_all_data[refined_hex_8_all_data['service_name'].isin(['Link'])]
hex_8_all_data.columns

Index(['city', 'service_name', 'pickup_cluster', 'pickup_hex_8',
       'fe_cus_count', 'fe_count', 'requested_orders', 'cobrm',
       'expiry_mapped', 'cobra', 'accepted_orders', 'ocara', 'net_orders',
       'aor', 'fe2rr', 'fe2net', 'taxi_high_income', 'taxi_medium_income',
       'taxi_low_income', 'link_only_service', 'auto_only_service',
       'both_service', 'link_ps', 'link_nps', 'auto_ps', 'auto_nps',
       'fe_intent_stable', 'fe_intent_increasing', 'fe_intent_declining'],
      dtype='object')

In [20]:
hex_8_all_data.describe()

,fe_cus_count,fe_count,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders,aor,fe2rr,fe2net,taxi_high_income,taxi_medium_income,taxi_low_income,link_only_service,auto_only_service,both_service,link_ps,link_nps,auto_ps,auto_nps,fe_intent_stable,fe_intent_increasing,fe_intent_declining
count,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1998.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000,1999.000000
mean,1025.482741,3759.492246,675.464232,2.057529,40.968984,131.326663,502.478739,111.851426,271.029515,62.354354,18.061531,5.417709,404.630815,272.687844,51.189095,98.815408,144.536268,645.574287,139.452726,245.380190,221.260130,363.391696,337.763882,403.431216,170.871436
std,2103.330224,8212.575206,1432.721151,3.915677,120.463644,314.321455,1026.933039,232.362412,557.164357,29.021819,8.953372,3.944067,894.232561,537.595619,96.525761,189.521964,292.914341,1383.512775,281.119674,520.950135,486.036235,773.491357,678.963273,848.749936,377.831765
min,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,62.500000,8.000000,0.000000,1.000000,1.000000,4.000000,1.000000,1.000000,50.000000,14.000000,2.000000,5.000000,5.000000,1.000000,3.000000,2.000000,8.000000,2.000000,3.000000,2.000000,4.000000,5.000000,6.000000,2.000000
50%,143.000000,453.000000,78.000000,1.000000,5.000000,13.000000,57.000000,16.000000,26.000000,73.000000,18.000000,6.000000,43.000000,39.000000,8.000000,19.000000,18.000000,70.000000,18.000000,28.000000,21.000000,40.000000,43.000000,49.000000,18.000000
75%,1090.500000,3643.000000,675.500000,3.000000,21.000000,101.000000,537.000000,118.000000,287.000000,82.000000,21.000000,8.000000,391.000000,308.500000,61.000000,118.000000,152.500000,655.500000,151.000000,244.500000,211.000000,364.000000,367.500000,415.000000,161.000000
max,25922.000000,100666.000000,14122.000000,77.000000,1368.000000,2937.000000,10310.000000,2898.000000,5596.000000,100.000000,100.000000,50.000000,12122.000000,6695.000000,1232.000000,2966.000000,3669.000000,17808.000000,3152.000000,6463.000000,6398.000000,10067.000000,8480.000000,10943.000000,4981.000000


## <center> Adding new required columns </center>

In [21]:
hex_8_all_data['high_income_%'] = round(hex_8_all_data['taxi_high_income']*100/hex_8_all_data['fe_cus_count'])
hex_8_all_data['high_income_thrshld'] = hex_8_all_data['high_income_%'].median()

hex_8_all_data['link_only_service_%'] = round(hex_8_all_data['link_only_service']*100/hex_8_all_data['fe_cus_count'])
hex_8_all_data['auto_only_service_%'] = round(hex_8_all_data['auto_only_service']*100/hex_8_all_data['fe_cus_count'])

hex_8_all_data['link_ps_%'] = round(hex_8_all_data['link_ps']*100/hex_8_all_data['fe_cus_count'])
hex_8_all_data['link_ps_threshld'] = hex_8_all_data['link_ps_%'].median()

hex_8_all_data['affluence_hi_tag'] = np.where(
                                        hex_8_all_data['high_income_%'] <= hex_8_all_data['high_income_thrshld'] , 
                                        'Less', 
                                        'High')
hex_8_all_data['service_aff_tag'] = np.where(
                                        hex_8_all_data['link_only_service_%'] > hex_8_all_data['auto_only_service_%'] , 
                                        'Yes', 
                                        'No')
hex_8_all_data['link_ps_tag'] = np.where(
                                        hex_8_all_data['link_ps_%'] <= hex_8_all_data['link_ps_threshld'] , 
                                        'Less', 
                                        'High')

In [22]:
hex_8_all_data.head(5)

,city,service_name,pickup_cluster,pickup_hex_8,fe_cus_count,fe_count,requested_orders,cobrm,expiry_mapped,cobra,accepted_orders,ocara,net_orders,aor,fe2rr,fe2net,taxi_high_income,taxi_medium_income,taxi_low_income,link_only_service,auto_only_service,both_service,link_ps,link_nps,auto_ps,auto_nps,fe_intent_stable,fe_intent_increasing,fe_intent_declining,high_income_%,high_income_thrshld,link_only_service_%,auto_only_service_%,link_ps_%,link_ps_threshld,affluence_hi_tag,service_aff_tag,link_ps_tag
0,Bangalore,Link,Hoskote_BLR,8861893893fffff,40,48,2,0,0,0,0,0,0,0.0,4.0,0.0,10,8,0,5,7,15,4,6,3,7,10,11,4,25.0,32.0,12.0,18.0,10.0,13.0,Less,No,Less
2,Bangalore,Link,Attibele,8861893713fffff,112,268,21,2,1,10,8,4,2,38.0,8.0,1.0,23,20,5,12,7,35,7,9,13,13,25,24,4,21.0,32.0,11.0,6.0,6.0,13.0,Less,Yes,Less
3,Bangalore,Link,Attibele,88618936a7fffff,4,22,2,0,0,2,0,0,0,0.0,9.0,0.0,1,1,0,1,0,2,0,0,0,1,1,2,0,25.0,32.0,25.0,0.0,0.0,13.0,Less,Yes,Less
5,Bangalore,Link,Sarjapur,88618935ebfffff,11,57,9,4,0,0,4,2,2,44.0,16.0,4.0,5,1,2,1,1,7,1,3,0,2,2,4,3,45.0,32.0,9.0,9.0,9.0,13.0,High,No,Less
7,Bangalore,Link,Sarjapur,88618935e9fffff,1,1,1,0,1,0,1,0,0,100.0,100.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,32.0,0.0,0.0,0.0,13.0,Less,No,Less


In [23]:
## Filtering required columns

df_hex_8_all_data = hex_8_all_data [[
                                        'city', 'pickup_cluster', 'pickup_hex_8', 
                                        'fe_count', 'requested_orders', 'net_orders',
                                        'high_income_%', 'high_income_thrshld',
                                        'link_only_service_%',  'auto_only_service_%',
                                        'link_ps_%', 'link_ps_threshld', 
                                        'affluence_hi_tag', 'service_aff_tag', 'link_ps_tag'
                                    ]]

df_hex_8_all_data['flag'] = np.where(
                                    df_hex_8_all_data['affluence_hi_tag'].isin(['Less'])
                                    &
                                    df_hex_8_all_data['service_aff_tag'].isin(['Yes'])
                                    &
                                    df_hex_8_all_data['link_ps_tag'].isin(['Less']),
                                    'Low',
                                    'High'
                                    )

In [24]:
## Final data

df_hex_8_all_data

,city,pickup_cluster,pickup_hex_8,fe_count,requested_orders,net_orders,high_income_%,high_income_thrshld,link_only_service_%,auto_only_service_%,link_ps_%,link_ps_threshld,affluence_hi_tag,service_aff_tag,link_ps_tag,flag
0,Bangalore,Hoskote_BLR,8861893893fffff,48,2,0,25.0,32.0,12.0,18.0,10.0,13.0,Less,No,Less,High
2,Bangalore,Attibele,8861893713fffff,268,21,2,21.0,32.0,11.0,6.0,6.0,13.0,Less,Yes,Less,Low
3,Bangalore,Attibele,88618936a7fffff,22,2,0,25.0,32.0,25.0,0.0,0.0,13.0,Less,Yes,Less,Low
5,Bangalore,Sarjapur,88618935ebfffff,57,9,2,45.0,32.0,9.0,9.0,9.0,13.0,High,No,Less,High
7,Bangalore,Sarjapur,88618935e9fffff,1,1,0,0.0,32.0,0.0,0.0,0.0,13.0,Less,No,Less,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4017,Bangalore,Bidadi,886014413dfffff,12,1,0,60.0,32.0,0.0,20.0,20.0,13.0,High,No,High,High
4021,Bangalore,Bidadi,8860144135fffff,60,1,0,34.0,32.0,7.0,76.0,0.0,13.0,High,No,Less,High
4022,Bangalore,Bidadi,8860144125fffff,15,2,0,0.0,32.0,0.0,0.0,0.0,13.0,Less,No,Less,High
4024,Bangalore,Bidadi,8860144113fffff,3,2,0,0.0,32.0,0.0,0.0,0.0,13.0,Less,No,Less,High


In [25]:
cluster_high_low = df_hex_8_all_data.groupby(['pickup_cluster','affluence_hi_tag', 
                                            'service_aff_tag','link_ps_tag']) \
                                            .agg({ 'pickup_hex_8' : 'nunique'}).reset_index()
cluster_high_low

,pickup_cluster,affluence_hi_tag,service_aff_tag,link_ps_tag,pickup_hex_8
0,AECS Layout Brookefield,High,No,High,1
1,Adugodi,High,No,Less,2
2,Adugodi Traffic Station,High,No,High,1
3,Agara Lake,High,No,High,2
4,Akshaynagar,High,No,High,1
...,...,...,...,...,...
592,Yeshwanthpur,High,No,Less,2
593,Yeshwanthpur,High,Yes,Less,1
594,Yeshwanthpur,Less,Yes,High,2
595,Yeshwanthpur,Less,Yes,Less,1


In [26]:
df_hex_8_all_data.groupby(['flag']) \
                        .agg({ 'pickup_hex_8' : 'nunique', 
                               'fe_count' : 'sum', 
                               'requested_orders' : 'sum',
                               'net_orders' : 'sum'
                             }).reset_index()

,flag,pickup_hex_8,fe_count,requested_orders,net_orders
0,High,1659,7254641,1309648,527143
1,Low,340,260584,40605,14645
